In [ ]:
import numpy as np

In [ ]:
def get_border(array):
    top = array[0, :]
    bottom = array[-1, :]
    left = array[:, 0]
    right = array[:, -1]
    return top, bottom, left, right

In [15]:
tiles = {}
with open('day20_input.txt') as f:
    data = f.read().split('\n\n')

In [16]:
for line in data:
    tile_num, tile_data = line.split(':\n')
    rows = len(tile_data.splitlines())
    tiles[int(tile_num.split()[1])] = np.array([list(map(int, tile_data.replace('.', '0').replace('#', '1').split('\n')[row])) for row in range(rows) ])

In [ ]:
len(tiles.keys())

In [ ]:
result = 1
neighbors = {}
corners = []
edge = []
middle = []
tile_list = list(tiles.keys())
for tile in tile_list:
    #print(tile)
    top, bottom, left, right = get_border(tiles[tile])
    check_tiles = list(tiles.keys())
    idx = check_tiles.index(tile)
    check_tiles.pop(idx)
    #print(check_tiles)
    sides = []
    neighbor = []
    for other_tile in check_tiles:
        same_sides = 0
        for vals in list(get_border(tiles[other_tile])):
            if (sum(abs(top - vals)) == 0) | (sum(abs(top - vals[::-1])) == 0):
                same_sides += 1
                neighbor.append(other_tile)
            if (sum(abs(bottom - vals)) == 0) | (sum(abs(bottom - vals[::-1])) == 0):
                same_sides += 1
                neighbor.append(other_tile)
            if (sum(abs(left - vals)) == 0) | (sum(abs(left - vals[::-1])) == 0):
                same_sides += 1
                neighbor.append(other_tile)
            if (sum(abs(right - vals)) == 0) | (sum(abs(right - vals[::-1])) == 0):
                same_sides += 1
                neighbor.append(other_tile)
        sides.append(same_sides)
        
    neighbors[tile] = neighbor
    if sum(sides) == 2:
        result *= tile
        corners.append(tile)
    elif sum(sides) == 3:
        edge.append(tile)
    else:
        middle.append(tile)
                
print(result)

In [ ]:
neighbors

In [ ]:
neighbors[neighbors[corners[0]][1]]

In [ ]:
tiles[corners[0]]

In [ ]:
np.rot90(tiles[corners[0]], k=1, axes=(1,0))

In [ ]:
puzzle = np.zeros((12, 12), int)
puzzle[0, 0] = corners[0]

In [ ]:
possibilities = [first_tile,
                 np.rot90(first_tile, 1, axes=(1, 0)),
                 np.rot90(first_tile, 2, axes=(1, 0)),
                 np.rot90(first_tile, 3, axes=(1, 0)),
                 np.flip(first_tile, axis=1),
                 np.rot90(np.flip(first_tile, axis=1), 1, axes=(1, 0)),
                 np.rot90(np.flip(first_tile, axis=1), 2, axes=(1, 0)),
                 np.rot90(np.flip(first_tile, axis=1), 3, axes=(1, 0))]

In [ ]:
corner = 3
first_tile = tiles[corners[corner]]
possibilities = [first_tile,
                 np.rot90(first_tile, 1, axes=(1, 0)),
                 np.rot90(first_tile, 2, axes=(1, 0)),
                 np.rot90(first_tile, 3, axes=(1, 0)),
                 np.flip(first_tile, axis=1),
                 np.rot90(np.flip(first_tile, axis=1), 1, axes=(1, 0)),
                 np.rot90(np.flip(first_tile, axis=1), 2, axes=(1, 0)),
                 np.rot90(np.flip(first_tile, axis=1), 3, axes=(1, 0))]
edge1 = get_border(tiles[neighbors[corners[corner]][0]])
edge2 = get_border(tiles[neighbors[corners[corner]][1]])
for orientation in possibilities:
    _, bottom, _, right = get_border(orientation)
    check_bottom1 = [sum(abs(edge1[i] - bottom)) for i in range(4)]
    check_bottom2 = [sum(abs(edge2[i] - bottom)) for i in range(4)]
    check_right1 = [sum(abs(edge1[i] - right)) for i in range(4)]
    check_right2 = [sum(abs(edge2[i] - right)) for i in range(4)]
    print(check_bottom1)
    print(check_bottom2)
    print(check_right1)
    print(check_right2)
    if (0 in check_bottom1) and (0 in check_right1):
        print('Found it')
    elif (0 in check_bottom2) and (0 in check_right2):
        print('Found it 2')
    print()

In [ ]:
0 in 

In [ ]:
first_tile = tiles[corners[0]]
top, bottom, left, right = get_border(first_tile)
neighbor_borders = list(get_border(tiles[other_tile]))
check_bottom = [sum(bottom-val) for val in neighbor_borders]
check_right = [sum(right-val) for val in neighbor_borders]

while (check_bottom[1] != 0) and (check_right[2] != 0):
    first_tile = np.rot90(first_tile, 1, axes=(1, 0))
    check_bottom = [sum(bottom-val) for val in neighbor_borders]
    check_right = [sum(right-val) for val in neighbor_borders]

In [ ]:
edge1

In [ ]:
np.flip(first_tile, axis=1)

In [ ]:
puzzle

In [ ]:
first_tile

In [ ]:
np.flip(first_tile, axis=1)

In [ ]:
np.rot90(np.flip(first_tile, axis=1), 1, axes=(1, 0))

In [ ]:
np.rot90(first_tile, 2, axes=(1, 0))

In [ ]:
np.rot90(first_tile, 3, axes=(1, 0))

In [18]:
from collections import deque
import math

import numpy as np
import scipy.ndimage


def parse(s):
    tiles = {}
    lookup = {'.': 0, '#': 1}
    for tile in s.split('\n\n'):
        tile_id, data = tile.split(':\n')
        tile_id = int(tile_id.split()[-1])
        arr = np.array([[lookup[c] for c in row.lstrip()] for row in data.split('\n')])
        arr.flags.writeable = False
        tiles[tile_id] = arr[::-1, :]
    return tiles


def edges(tile, flips=False):
    for s in [(slice(None), 0), (slice(None), -1), (0, slice(None)), (-1, slice(None))]:
        edge = tile[s]
        yield tuple(edge)
        if flips:
            yield tuple(edge)[::-1]


def fit_corner(tile, other_edges):
    # Make sure we put the edges in the right spot
    # If top edge is a match in any form, flip vertically
    if tuple(tile[0, :]) in other_edges or tuple(tile[0, ::-1]) in other_edges:
        tile = tile[::-1]
    # If left edge is a match in any form, flip horizontally
    if tuple(tile[:, 0]) in other_edges or tuple(tile[::-1, 0]) in other_edges:
        tile = tile[:, ::-1]
    return tile


def fit_above(tile, edge):
    # Make sure we put the matching edge at the top
    # If any version of left or right matches, rotate
    if edge in (tuple(tile[:, 0]), tuple(tile[::-1, 0]), tuple(tile[:, -1]), tuple(tile[::-1, -1])):
        tile = tile.T
    # If either top or bottom edge reversed is a match, flip horizontally
    if tuple(tile[0, ::-1]) == edge or tuple(tile[-1, ::-1]) == edge:
        tile = tile[:, ::-1]
    # If bottom edge is a match, flip vertically
    if tuple(tile[-1, :]) == edge:
        tile = tile[::-1]
    return tile


def fit_left(tile, edge):
    return fit_above(tile.T, edge).T


def find_match(pool, tiles, edge):
    while pool:
        choice = pool.pop()
        tile = tiles[choice]
        if edge in set(edges(tile, flips=True)):
            return choice
        pool.appendleft(choice)


def assemble(tiles):
    queue = deque(tiles)
    n = int(math.sqrt(len(tiles)))
    layout = [[None] * n for _ in range(n)]

    # Loop until we find the first corner
    while queue:
        tile_id = queue.pop()
        tile = tiles[tile_id]
        other_edges = {edge for other in tiles.values() for edge in edges(other) if other is not tile}
        count = sum(edge in other_edges for edge in edges(tile, flips=True))
        if count != 2:
            queue.appendleft(tile_id)
            continue

        tiles[tile_id] = fit_corner(tile, other_edges)
        layout[0][0] = tile_id
        break

    for row in range(n):
        if row > 0:
            match = tuple(tiles[layout[row - 1][0]][-1, :])
            choice = find_match(queue, tiles, match)
            fit = fit_above(tiles[choice], match)
            tiles[choice] = fit
            layout[row][0] = choice
        for col in range(1, n):
            match = tuple(tiles[layout[row][col-1]][:, -1])
            choice = find_match(queue, tiles, match)
            fit = fit_left(tiles[choice], match)
            tiles[choice] = fit
            layout[row][col] = choice

    return layout


def stitch(layout, tiles):
    return np.concatenate(
        tuple(np.concatenate(tuple(tiles[tile_id][1:-1, 1:-1] for tile_id in row), axis=1)
              for row in layout), axis=0)


def roughness(img):
    m = '''                  #.
#    ##    ##    ###
 #  #  #  #  #  #   '''
    monster = np.array([[1 if c == '#' else 0 for c in row] for row in m.split('\n')])
    locs = 0
    for i in range(8):
        kernel = monster
        if i & 0x1:
            kernel = kernel.T
        if i & 0x2:
            kernel = kernel[::-1]
        if i & 0x4:
            kernel = kernel[:, ::-1]
        con = scipy.ndimage.convolve(img, kernel, mode='constant')
        locs = (con == monster.sum()).sum()
        if locs:
            break

    return int(img.sum() - locs * monster.sum())

from aocd.models import Puzzle

t = '''Tile 2311:
..##.#..#.
##..#.....
#...##..#.
####.#...#
##.##.###.
##...#.###
.#.#.#..##
..#....#..
###...#.#.
..###..###
Tile 1951:
#.##...##.
#.####...#
.....#..##
#...######
.##.#....#
.###.#####
###.##.##.
.###....#.
..#.#..#.#
#...##.#..
Tile 1171:
####...##.
#..##.#..#
##.#..#.#.
.###.####.
..###.####
.##....##.
.#...####.
#.##.####.
####..#...
.....##...
Tile 1427:
###.##.#..
.#..#.##..
.#.##.#..#
#.#.#.##.#
....#...##
...##..##.
...#.#####
.#.####.#.
..#..###.#
..##.#..#.
Tile 1489:
##.#.#....
..##...#..
.##..##...
..#...#...
#####...#.
#..#.#.#.#
...#.#.#..
##.#...##.
..##.##.##
###.##.#..
Tile 2473:
#....####.
#..#.##...
#.##..#...
######.#.#
.#...#.#.#
.#########
.###.#..#.
########.#
##...##.#.
..###.#.#.
Tile 2971:
..#.#....#
#...###...
#.#.###...
##.##..#..
.#####..##
.#..####.#
#..#.#..#.
..####.###
..#.#.###.
...#.#.#.#
Tile 2729:
...#.#.#.#
####.#....
..#.#.....
....#..#.#
.##..##.#.
.#.####...
####.#.#..
##.####...
##..#.##..
#.##...##.
Tile 3079:
#.#.#####.
.#..######
..#.......
######....
####.#..#.
.#...#.##.
#.#####.##
..#.###...
..#.......
..#.###...'''


#tiles = parse(t)
#l = assemble(tiles)
#assert l[0][0] * l[0][-1] * l[-1][0] * l[-1][-1] == 20899048083289
#
#img = stitch(l, tiles)
#assert roughness(img) == 273

#puz = Puzzle(2020, 20)
#data = open('day20_input.txt').read()
#print(data)
#tiles = parse(data)
l = assemble(tiles)

answer_a = l[0][0] * l[0][-1] * l[-1][0] * l[-1][-1]
print(f'Part 1: {answer_a}')

img = stitch(l, tiles)
answer_b = roughness(img)
print(f'Part 2: {answer_b}')

Part 1: 30425930368573
Part 2: 2453
